In [19]:
import sys

# input comes from STDIN (standard input)
#a = "foo foo quux labs foo bar quux"
a=open("foo.txt")
for line in a:
    # remove leading and trailing whitespace
    line = line.strip()
    #print(line)
    # split the line into words
    words = line.split()
    #print(words)
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print ('%s\t%s' % (word, 1))
#         print (word+"\t"+ 1)
        
        f = open("imd.txt", "a")
        f.write('%s\t%s\n' % (word, 1))
                
        f.close()
        
        

foo	1
foo	1
quux	1
labs	1
foo	1
bar	1
quux	1


In [4]:
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

b=open("foo.txt") #dee

# input comes from STDIN
for line in b:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print ('%s\t%s' % (current_word, current_count))
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print ('%s\t%s' % (current_word, current_count))

ValueError: not enough values to unpack (expected 2, got 1)

In [17]:
import sys

def read_input(file):
    for line in file:
        # split the line into words
        yield line.split()

def main(separator='\t'):
    # input comes from STDIN (standard input)
    #a = "foo foo quux labs foo bar quux"
    a=open("foo.txt")
    data = read_input(a)
    for words in data:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        for word in words:
            print ('%s%s%d' % (word, separator, 1))
            f = open("imd.txt", "a")
            f.write('%s%s%d\n' % (word, separator, 1))
                
            f.close()

if __name__ == "__main__":
    main()

foo	1
foo	1
quux	1
labs	1
foo	1
bar	1
quux	1


In [18]:
"""A more advanced Reducer, using Python iterators and generators."""

from itertools import groupby
from operator import itemgetter
import sys

def read_mapper_output(file, separator='\t'):
    for line in file:
        yield line.rstrip().split(separator, 1)

def main(separator='\t'):
    # input comes from STDIN (standard input)
    
    b=open("imd.txt")
    data = read_mapper_output(b, separator=separator)
    
    #data = read_mapper_output(a)
    # groupby groups multiple word-count pairs by word,
    # and creates an iterator that returns consecutive keys and their group:
    #   current_word - string containing a word (the key)
    #   group - iterator yielding all ["&lt;current_word&gt;", "&lt;count&gt;"] items
    for current_word, group in groupby(data, itemgetter(0)):
        try:
            total_count = sum(int(count) for current_word, count in group)
            print ("%s%s%d" % (current_word, separator, total_count))
        except ValueError:
            # count was not a number, so silently discard this item
            print("except error")
            pass

if __name__ == "__main__":
    main()

foo	2
quux	1
labs	1
foo	1
bar	1
quux	1
foo	2
quux	1
labs	1
foo	1
bar	1
quux	1
foo	2
quux	1
labs	1
foo	1
bar	1
quux	1


In [5]:
import itertools

def map_reduce(i,mapper,reducer):
  intermediate = []
  for (key,value) in i.items():
    intermediate.extend(mapper(key,value))
  groups = {}
  for key, group in itertools.groupby(sorted(intermediate), lambda x: x[0]):
    groups[key] = list([y for x, y in group])
  return [reducer(intermediate_key,groups[intermediate_key]) for intermediate_key in groups] 

In [8]:
#word_count.py

import string
import map_reduce

def mapper(input_key,input_value):
  return [(word,1) for word in 
          remove_punctuation(input_value.lower()).split()]

def remove_punctuation(s):
  return s.translate(string.maketrans("",""), string.punctuation)

def reducer(intermediate_key,intermediate_value_list):
  return (intermediate_key,sum(intermediate_value_list))

filenames = ["text\a.txt"]
i = {}
for filename in filenames:
  f = open(filename)
  i[filename] = f.read()
  f.close()

print (map_reduce.map_reduce(i,mapper,reducer))

FileNotFoundError: [Errno 2] No such file or directory: 'text\x07.txt'